<a href="https://colab.research.google.com/github/sameer203/Tensorflow_/blob/main/11_NLP_fundamentals_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-11-03 10:21:02--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.128, 142.250.73.208, 142.250.65.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.04s   

2021-11-03 10:21:03 (15.4 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-11-03 10:21:03--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-11-03 10:21:03 (58.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys,unzip_data

In [4]:
unzip_data("nlp_getting_started.zip")

### Visualizing Test data
1. pandas
2. tensorflowload text

In [5]:
import pandas as pd

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# Shuffle training data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df_shuffled.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
len(train_df), len(test_df)

(7613, 3263)

In [11]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n {text}\n")

Target: 0 (not real diaster)
Text:
 Pandemonium In Aba As Woman Delivers Baby Without Face (Photos) - http://t.co/Ykdsp0nRDQ

Target: 1 (real diaster)
Text:
 @rvfriedmann Hell is just a fraction of his belief of total annihilation destruction of USA @LodiSilverado @ritzy_jewels

Target: 1 (real diaster)
Text:
 Islamic State group in Egypt threatens to kill Croat hostage http://t.co/VdgfXYX3bw

Target: 0 (not real diaster)
Text:
 @HomeworldGym @thisisperidot D: What? That's a tragedy. You have a wonderful nose

Target: 0 (not real diaster)
Text:
 Owner of Chicago-Area Gay Bar Admits to Arson Scheme http://t.co/UBFr1URAFc #LGBT | https://t.co/AlnV51d95x



In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(train_df_shuffled["text"].to_numpy(), 
                                                                              train_df_shuffled["target"].to_numpy(), 
                                                                              test_size=0.1, 
                                                                              random_state=42)

In [14]:
len(train_sentences), len(test_sentences)

(6851, 762)

In [15]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

### Text Vectorization

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [17]:
# use the defaultTextVectorization parameters
text_vectorization = TextVectorization(max_tokens=None,
                                       standardize="lower_and_strip_punctuation",
                                       split="whitespace",
                                       ngrams=None,
                                       output_mode="int",
                                       output_sequence_length=None,
                                       pad_to_max_tokens=False)


In [18]:
# Find the average number of tokens in the training tweets
sum = 0
for i in train_sentences:
  sum = sum + len(i.split())
round(sum)
average = round(sum/len(train_sentences))
average

15

In [19]:
# Setup text vectorization variables
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                       output_sequence_length=max_length,
                                       output_mode="int")

In [20]:
# Fit the text vectorizer to the training data
text_vectorizer.adapt(train_sentences)

In [21]:
# Create a samplesentence and tokenize it
sample_sentence= "My name is sameer"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 13, 735,   9,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# Vectorize a random sentence from training set
random_sentence = random.choice(train_sentences)

random_sentence, text_vectorizer([random_sentence])

('Pic of 16yr old PKK suicide bomber who detonated bomb in Turkey Army trench released http://t.co/uWfvWMjepU http://t.co/pxONlrqUsm',
 <tf.Tensor: shape=(1, 15), dtype=int64, numpy=
 array([[618,   6, 634, 206, 534,  87, 189,  65, 551, 108,   4, 613, 164,
         756, 342]])>)

In [23]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab[:5]

['', '[UNK]', 'the', 'a', 'in']

In [24]:
words_in_vocab[-5:]

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [25]:
from tensorflow.keras import layers

In [26]:
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)
embedding

In [27]:
# turn sentences into dense vector of fixed size
random_sentence = random.choice(train_sentences)

sample_embed = embedding(text_vectorizer([random_sentence]))

random_sentence, sample_embed

('today will be another dualcom with @Im_Razed !!! if you enjoyed yesterdays check out todays at 5 pm easter!!!',
 <tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[-0.02633812, -0.01662265,  0.01160215, ...,  0.0318576 ,
          -0.03730651, -0.03813658],
         [-0.01614846,  0.04060316,  0.03894493, ..., -0.03931365,
          -0.04026083, -0.00149997],
         [ 0.00856771, -0.00198931,  0.0054104 , ...,  0.03122773,
           0.04063779,  0.04118487],
         ...,
         [-0.04068559, -0.02890112,  0.02697224, ..., -0.04217473,
          -0.03300737, -0.03859389],
         [-0.00972767, -0.03914456,  0.03216727, ..., -0.03195379,
          -0.02598461,  0.01492815],
         [-0.0239874 , -0.0129335 ,  0.03573452, ...,  0.03974927,
           0.03264448,  0.01242129]]], dtype=float32)>)

In [28]:
# checkout single tokens embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-2.6338125e-02, -1.6622651e-02,  1.1602152e-02,  1.6781315e-03,
         3.9639805e-02,  3.9585892e-02,  4.4506524e-02,  4.6018664e-02,
        -4.5261502e-02,  2.8976250e-02, -3.1017734e-02,  2.8895702e-02,
         1.9785907e-02, -7.3748827e-04, -3.5221875e-02, -2.0689761e-02,
         1.7220888e-02,  2.3482118e-02,  2.1418322e-02, -2.5334246e-03,
         8.6969733e-03, -4.9985398e-02,  1.2247182e-02,  3.0190349e-03,
        -9.2577189e-05, -3.8882755e-02, -1.5999578e-02,  1.3230816e-03,
        -3.2123972e-02, -3.4069739e-02, -2.0396225e-03, -3.7255466e-02,
         1.3575878e-02, -1.5592646e-02,  1.0152627e-02,  1.4707450e-02,
         1.4021326e-02, -7.7592731e-03, -3.9106235e-03, -2.5365686e-02,
        -2.4902618e-02,  2.8353121e-02,  2.5816035e-02, -2.7458692e-02,
         4.4737581e-02, -9.2223175e-03, -8.9241490e-03, -5.9266016e-04,
         4.4479396e-02,  4.0381178e-03, -2.8462648e-02, -1.8623400e-02,
        -2.2988

### Model 0: Getting a baseline

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


In [30]:
# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [31]:
# Evaluate the baseline model
baseline_score = model_0.score(test_sentences, test_labels)
baseline_score*100

79.26509186351706

In [32]:
baseline_preds = model_0.predict(test_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [33]:
# Function to evaluate : Accuracy, precision, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_result = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_result

In [34]:
baseline_results = calculate_result(test_labels, baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: Asimple dense model

In [35]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [36]:
# Build model with functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [37]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
dense (Dense)                (None, 15, 1)             129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compile the model
model_1.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1.fit(x=train_sentences,
            y=train_labels,
            epochs=5,
            validation_data=(test_sentences, test_labels),
            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                   experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20211103-102107
Epoch 1/5
215/215 [==============================] - 4s 15ms/step - loss: 0.6499 - accuracy: 0.6384 - val_loss: 0.6314 - val_accuracy: 0.6490
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.5969 - accuracy: 0.6892 - val_loss: 0.6255 - val_accuracy: 0.6480
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.5769 - accuracy: 0.6915 - val_loss: 0.6284 - val_accuracy: 0.6463
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.5682 - accuracy: 0.6907 - val_loss: 0.6310 - val_accuracy: 0.6481
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.5633 - accuracy: 0.6920 - val_loss: 0.6324 - val_accuracy: 0.6506


In [39]:
model_1.evaluate(test_sentences, test_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.6324 - accuracy: 0.6506


[0.6324405670166016, 0.6505686640739441]

In [40]:
model_1_pred_probs = model_1.predict(test_sentences)
model_1_pred_probs

array([[[0.43902552],
        [0.43902552],
        [0.43902552],
        ...,
        [0.03999263],
        [0.43902552],
        [0.36578104]],

       [[0.58603036],
        [0.40241712],
        [0.44122934],
        ...,
        [0.8105322 ],
        [0.43902552],
        [0.62105215]],

       [[0.93338466],
        [0.6009251 ],
        [0.6007167 ],
        ...,
        [0.9725593 ],
        [0.6007167 ],
        [0.58063936]],

       ...,

       [[0.58603036],
        [0.29955712],
        [0.40241712],
        ...,
        [0.8105322 ],
        [0.43902552],
        [0.36578104]],

       [[0.09752125],
        [0.5645426 ],
        [0.4039295 ],
        ...,
        [0.3379818 ],
        [0.35784066],
        [0.3736918 ]],

       [[0.17351976],
        [0.15833935],
        [0.40391567],
        ...,
        [0.34737954],
        [0.24446484],
        [0.6374274 ]]], dtype=float32)

In [41]:
model_1_pred_probs.shape

(762, 15, 1)

In [42]:
#adding Global average pooling layer to offset the error
# Build model with functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

# Compile the model
model_1.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1.fit(x=train_sentences,
            y=train_labels,
            epochs=5,
            validation_data=(test_sentences, test_labels),
            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                   experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20211103-102123
Epoch 1/5
215/215 [==============================] - 4s 14ms/step - loss: 0.5561 - accuracy: 0.7803 - val_loss: 0.5196 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3908 - accuracy: 0.8511 - val_loss: 0.4604 - val_accuracy: 0.7900
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3126 - accuracy: 0.8824 - val_loss: 0.4484 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2611 - accuracy: 0.9037 - val_loss: 0.4553 - val_accuracy: 0.7940
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2226 - accuracy: 0.9191 - val_loss: 0.4697 - val_accuracy: 0.7835


In [43]:
model_1.evaluate(test_sentences, test_labels)

24/24 [==============================] - 0s 1ms/step - loss: 0.4697 - accuracy: 0.7835


[0.46973225474357605, 0.7834645509719849]

In [44]:
model_1_pred_probs = model_1.predict(test_sentences)
model_1_pred_probs[:10]

array([[0.29385564],
       [0.79328823],
       [0.99729156],
       [0.15011382],
       [0.10198486],
       [0.94665945],
       [0.88179946],
       [0.99644166],
       [0.9691912 ],
       [0.27975604]], dtype=float32)

In [45]:
# convert model prediction probability to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [46]:
model_1_results = calculate_result(y_true=test_labels,
                                   y_pred = model_1_preds)
model_1_results

{'accuracy': 78.34645669291339,
 'f1': 0.780192419920975,
 'precision': 0.7888731352165597,
 'recall': 0.7834645669291339}

In [47]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [48]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

### Visualizing learned embeddings

In [49]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [51]:
# Model summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [59]:
# Get the weight matrix of embedding layer
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights, embed_weights[0].shape

(array([[-0.00749328, -0.00060301, -0.01241952, ..., -0.06877372,
          0.02373117,  0.05245439],
        [-0.00398308, -0.03966501, -0.01457286, ..., -0.02948026,
         -0.01524954,  0.06182721],
        [ 0.02371028,  0.02014341, -0.05458614, ..., -0.03240985,
         -0.03208706,  0.07097179],
        ...,
        [-0.00984229,  0.01481536,  0.04407943, ...,  0.04051197,
          0.03055767,  0.00885681],
        [ 0.0897963 , -0.03927771, -0.10609651, ..., -0.07957483,
          0.08100583,  0.05091857],
        [ 0.07932884, -0.11056592, -0.06926775, ..., -0.12497937,
          0.07169607,  0.12949784]], dtype=float32), (128,))

In [60]:
# Create embedding files
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [62]:
# downloadfiles
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Recurrent Neural Network (RNN)

In [66]:
# LSTM model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
#x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name="Model_2_LSTM")

In [67]:
model_2.summary()

Model: "Model_2_LSTM"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [68]:
# compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [69]:
#Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(test_sentences, test_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20211103-113232
Epoch 1/5
215/215 [==============================] - 8s 28ms/step - loss: 0.1937 - accuracy: 0.9345 - val_loss: 0.6512 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.1333 - accuracy: 0.9480 - val_loss: 0.6491 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 5s 25ms/step - loss: 0.1069 - accuracy: 0.9602 - val_loss: 0.7840 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 5s 25ms/step - loss: 0.0883 - accuracy: 0.9661 - val_loss: 0.7510 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 5s 25ms/step - loss: 0.0709 - accuracy: 0.9724 - val_loss: 0.9030 - val_accuracy: 0.7677


In [70]:
model_2_pred_probs = model_2.predict(test_sentences)
model_2_pred_probs[:10]

array([[1.9600391e-03],
       [7.0218009e-01],
       [9.9784964e-01],
       [1.4183652e-01],
       [2.3254752e-04],
       [9.6835476e-01],
       [6.1460316e-02],
       [9.9956769e-01],
       [9.9890900e-01],
       [3.2807809e-01]], dtype=float32)

In [71]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [72]:
model_2_results = calculate_result(test_labels, model_2_preds)
model_2_results

{'accuracy': 76.77165354330708,
 'f1': 0.7623720957471439,
 'precision': 0.7774904318981964,
 'recall': 0.7677165354330708}

In [73]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}